## History

Transformer architecture was introduced in 2017. The most influential models were:
- GPT, BERT (2018)
- GPT-2, DistilBERT, BART, T5 (2019)
- GPT-3 (2020) -> allows zero-shot learning

Different kinds of Transformer models:
- GPT-like (also called auto-regressive Transformer models)
- BERT-like (also called auto-encoding Transformer models)
- BART/T5-like (also called sequence-to-sequence Transformer models)


## Training

<u>Self-supervised</u><br>
Self-supervised learning is a type of training in which the objective is automatically computed from the inputs of the model. That means that humans are not needed to label the data! This type of model develops a statistical understanding of the language it has been trained on, but it’s not very useful for specific practical tasks.
Requires <u>transfer learning</u> on labelled data.

<u>Pretraining</u><br>
Training a model from scratch. Weights are randomly initialized.

<u>Transfer learning</u><br>
Fine-tune a pretrained model. Requires less data than training from scratch. Often resulting in better results. Replace the Head of the pretrained model (e.g. change number of output labels).

## Architecture
### Encoder-only
The encoder receives an input and creates a feature vector/tensor.

Good for natural language understanding (NLU).
Used for tasks:
- Sentence classification
- Named entity recognition
- Question answering
- Fill mask
- Sentiment analysis

Attention layers can access all the words in the initial sentence ("bi-directional" attention). Are often called auto-encoding models. Good at extracting meaningful information. Good at obtaining an understanding of sequences; and the relationship/interdependence between words.

Models include:
- ALBERT, BERT, DistilBERT, ELECTRA, RoBERTa

### Decoder-only
Receives an input and generates a feature vector/tensor (sequence). Can perform most of the same tasks as an Encoder, but with worse performance.

Good for causal language modeling or natural language generation (NLG).
Used for tasks:
- Text generation

Right context of a word is "masked". Attention layers can only access the words positioned before in the sentence ("uni-directional" attention). Are often called auto-regressive models. Use their past outputs as new inputs for the next output.

Models include:
- CTRL, GPT, GPT-2, Transformer XL


### Encoder-Decoder (Transformer)

Hyperparameters:
- Nx: Number of Encoder blocks
- Positional Encoding: How the positional encodings are created.

Used for tasks:
- Translation
- Summarization

Also called sequence-to-sequence models. Can perform the tasks of encoder and decoder models, but usually involves more complex tasks. Best for generating new sentences depending on a given input.

Models include:
- BART, mBART, Marian, T5

<img src="images/transformer_architecture.png" style="width:400px">

Junctions for the input prevent vanishing gradients through multiple layers.

<u>Positional Encoding</u>:<br>
Attention does not care about the order of the input. Positional encoding tries to fix that problem. Input-Encodings and Positional Encodings are added together. 
- Can be learned.
- Can use a function. 

<u>Scaled Dot-Product Attention</u>:<br>
Similar to attention mechanism. But uses an additional scaling step for numeric stability during gradient descent.

<img src="images/scaled_dot_product_attention.png" style="width:200px">

<u>Multi-Head Attention</u>:<br>
Also uses Values, Keys and Queries.

## Imports

In [ ]:
from transformers import pipeline

## Pipelines
Consists of three different stages:<br>
Tokenizer -> Model -> Postprocessing

### Sentiment analysis

In [ ]:
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
classifier("I've been waiting for a HuggingFace course my whole life.")

In [ ]:
classifier(["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"])

### Zero-shot-classification

Any amount of new labels can be provided. The model returns the probabilities for each label. No fine-tune needed.

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business", "data science"],
)

### Text generation

In [ ]:
generator = pipeline("text-generation", model="gpt2")
generator("In this course, we will teach you how to", num_return_sequences=3, max_length=30)

A specific model can be selected.

In [ ]:
generator = pipeline("text-generation", model="sberbank-ai/mGPT")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=1,
)

### Mask-filling

In [ ]:
unmasker = pipeline("fill-mask", model="distilbert-base-uncased")
unmasker("This course will teach you all about [MASK] models.", top_k=4)

### Named-Entity-Recognition

In [ ]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

### Question answering

In [ ]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

### Translation

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.", min_length=50)

## Tokenizer & Model & Postprocessing
### Tokenizer

- Split text into tokens (words, part of words, punctuation symbols)
- Add special tokens
- Match each token with unique ID in vocabulary in pretrained model

Different forms of tokenizers:
- Word-based have ids for words. Can grow very large or have many unknown words.
- Character-based have ids for characters. Smaller than word-based, but characters do not hold as much information as words. More tokens are created and limit the input length.
- Subword-based mixture of word- and character-based. Split words into meaningful subwords.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
    "Why are you so [MASK]?",
]

In [ ]:
tuple(raw_inputs)

In [ ]:
tokenizer.tokenize(raw_inputs[0])

In [ ]:
tokenizer.tokenize(raw_inputs[-1])

In [ ]:
inputs = tokenizer(raw_inputs, padding=True, truncation=True, max_length=8, return_tensors="pt")
inputs["input_ids"]

In [ ]:
tokenizer.decode(inputs["input_ids"][-1])

In [ ]:
tokenizer.pad_token_id

In [ ]:
inputs["attention_mask"]

### Model

- Convert Token IDs into embeddings
- Attention mechanism layers produce hidden state (high dimensionality)
- Head projects the hidden state into a different dimension


In [ ]:
from transformers import AutoModel

In [ ]:
model = AutoModel.from_pretrained(checkpoint)
outputs = model(**inputs)

In [ ]:
outputs.last_hidden_state.shape

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
outputs = model(**inputs)
outputs.logits.shape

In [ ]:
outputs.logits

### Postprocessing

In [ ]:
import torch

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=1)
predictions

In [ ]:
model.config.id2label

## Model configuration
- Load Model architecture from pretrained models

In [ ]:
from transformers import AutoConfig, DistilBertModel, DistilBertForSequenceClassification

- Change the configs

In [ ]:
distil_bert_config = AutoConfig.from_pretrained(checkpoint, n_layers=2)
distil_bert_config

- Initialize model with config architecture, but random weights

In [ ]:
distil_bert_model = DistilBertModel(distil_bert_config)

In [ ]:
distil_bert_model.save_pretrained("test_model")

In [ ]:
DistilBertModel.from_pretrained("test_model")

In [ ]:
DistilBertForSequenceClassification.from_pretrained("test_model")